In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

# Import from Part 2
from part2_enhanced_alice import KeyConditionedAttention, PositionalEncoding

# ============ Decoder Block with Key Conditioning ============
class DecoderBlock(nn.Module):
    """Transformer decoder block - must have correct keys to decrypt"""
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderBlock, self).__init__()

        # Self-attention on encrypted input
        self.self_attn = KeyConditionedAttention(d_model, num_heads)

        # Feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )

        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Key mixing layer (decryption depends on keys)
        self.key_mixer = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Linear(d_model, d_model)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, key_features):
        """
        x: (batch, seq_len, d_model) - encrypted sequence
        key_features: (batch, d_model) - decryption keys
        """
        # Self-attention with key conditioning
        attn_out = self.self_attn(x, x, x, key_features)
        x = self.norm1(x + self.dropout(attn_out))

        # Mix with key features (decryption step)
        batch_size, seq_len, _ = x.size()
        key_expanded = key_features.unsqueeze(1).expand(-1, seq_len, -1)
        x_with_key = torch.cat([x, key_expanded], dim=-1)
        key_mixed = self.key_mixer(x_with_key)
        x = x + key_mixed

        # Feed-forward
        ffn_out = self.ffn(x)
        x = self.norm2(x + ffn_out)

        return x


# ============ Enhanced Bob Decoder ============
class EnhancedBobDecoder(nn.Module):
    """
    Bob: Decrypts encrypted messages using correct keys
    """
    def __init__(self, vocab_size=256, d_model=256, num_heads=8,
                 num_layers=6, d_ff=1024, max_len=512, dropout=0.1):
        super(EnhancedBobDecoder, self).__init__()

        self.d_model = d_model
        self.vocab_size = vocab_size

        # Input projection
        self.input_projection = nn.Linear(d_model, d_model)

        # Positional encoding
        self.pos_encoding = PositionalEncoding(d_model, max_len)

        # Key encoder (must match Alice's key encoding)
        self.key_encoder = nn.Sequential(
            nn.Linear(512, d_model * 2),
            nn.LayerNorm(d_model * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 2, d_model),
            nn.LayerNorm(d_model),
            nn.Tanh()
        )

        # Stack of decoder blocks
        self.decoder_blocks = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

        # Output head to predict characters
        self.output_head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, vocab_size)
        )

        self.dropout = nn.Dropout(dropout)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, encrypted_sequence, key_sequences):
        """
        encrypted_sequence: (batch, seq_len, d_model) - from Alice
        key_sequences: same keys used for encryption
        """
        batch_size, seq_len, _ = encrypted_sequence.size()
        device = encrypted_sequence.device

        # Combine key sequences
        if isinstance(key_sequences[0], np.ndarray):
            key_combined = np.concatenate([ks[:512] for ks in key_sequences[:1]], axis=0)
            key_tensor = torch.FloatTensor(key_combined).to(device)
        else:
            key_combined = torch.cat([ks[:512] for ks in key_sequences[:1]], dim=0).to(device)
            key_tensor = key_combined

        # Expand for batch
        key_tensor = key_tensor.unsqueeze(0).expand(batch_size, -1)

        # Encode keys (same as Alice)
        key_features = self.key_encoder(key_tensor)

        # Project encrypted input
        x = self.input_projection(encrypted_sequence)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # Pass through decoder blocks with key conditioning
        for decoder_block in self.decoder_blocks:
            x = decoder_block(x, key_features)

        # Predict characters
        logits = self.output_head(x)

        return logits


# ============ Enhanced Eve Attacker ============
class EnhancedEveAttacker(nn.Module):
    """
    Eve: Tries to decrypt WITHOUT keys (adversary)
    Uses a different architecture to avoid copying Alice/Bob
    """
    def __init__(self, vocab_size=256, d_model=256, num_heads=8,
                 num_layers=8, d_ff=1024, max_len=512, dropout=0.3):
        super(EnhancedEveAttacker, self).__init__()

        self.d_model = d_model

        # Input projection
        self.input_projection = nn.Sequential(
            nn.Linear(d_model, d_model * 2),
            nn.LayerNorm(d_model * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 2, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU()
        )

        # Positional encoding
        self.pos_encoding = PositionalEncoding(d_model, max_len)

        # Powerful transformer layers (Eve uses more layers)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Output head
        self.output_head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.LayerNorm(d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2),
            nn.LayerNorm(d_model // 2),
            nn.GELU(),
            nn.Linear(d_model // 2, vocab_size)
        )

        self.dropout = nn.Dropout(dropout)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, encrypted_sequence):
        """
        encrypted_sequence: (batch, seq_len, d_model)
        No keys provided - Eve must break it without keys
        """
        # Project input
        x = self.input_projection(encrypted_sequence)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # Transform
        x = self.transformer(x)

        # Predict characters
        logits = self.output_head(x)

        return logits


# ============ Bob Decryption Module ============
class BobStringDecryption:
    """Wrapper for Bob decoder with string support"""
    def __init__(self, vocab_size=256, d_model=256, num_heads=8,
                 num_layers=6, max_len=512,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.bob = EnhancedBobDecoder(
            vocab_size=vocab_size,
            d_model=d_model,
            num_heads=num_heads,
            num_layers=num_layers,
            max_len=max_len
        ).to(device)

        from part1_enhanced_dcgan import StringPreprocessor
        self.preprocessor = StringPreprocessor(max_length=max_len)

    def decrypt_to_string(self, encrypted_sequence, key_sequences):
        """
        Decrypt encrypted sequence back to string
        encrypted_sequence: (batch, seq_len, d_model)
        key_sequences: same keys used for encryption
        """
        self.bob.eval()
        with torch.no_grad():
            logits = self.bob(encrypted_sequence, key_sequences)
            predicted_tokens = torch.argmax(logits, dim=-1)

        # Convert tokens to strings
        decrypted_strings = self.preprocessor.batch_tensor_to_strings(predicted_tokens)

        return decrypted_strings, predicted_tokens, logits


# ============ Eve Attack Module ============
class EveStringAttack:
    """Wrapper for Eve attacker with string support"""
    def __init__(self, vocab_size=256, d_model=256, num_heads=8,
                 num_layers=8, max_len=512,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.eve = EnhancedEveAttacker(
            vocab_size=vocab_size,
            d_model=d_model,
            num_heads=num_heads,
            num_layers=num_layers,
            max_len=max_len
        ).to(device)

        from part1_enhanced_dcgan import StringPreprocessor
        self.preprocessor = StringPreprocessor(max_length=max_len)

    def attack_to_string(self, encrypted_sequence):
        """
        Try to decrypt without keys
        encrypted_sequence: (batch, seq_len, d_model)
        """
        self.eve.eval()
        with torch.no_grad():
            logits = self.eve(encrypted_sequence)
            predicted_tokens = torch.argmax(logits, dim=-1)

        # Convert tokens to strings
        attacked_strings = self.preprocessor.batch_tensor_to_strings(predicted_tokens)

        return attacked_strings, predicted_tokens, logits


# ============ Usage Example ============
if __name__ == "__main__":
    print("="*70)
    print("ENHANCED BOB DECODER & EVE ATTACKER")
    print("="*70)

    # Initialize
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bob_dec = BobStringDecryption(device=device)
    eve_att = EveStringAttack(device=device)

    # Simulate encrypted sequence
    batch_size = 2
    seq_len = 256
    d_model = 256
    encrypted = torch.randn(batch_size, seq_len, d_model).to(device)

    # Simulate keys
    key_sequences = [np.random.rand(512) for _ in range(8)]

    print("\nBob attempting decryption with correct keys...")
    bob_strings, bob_tokens, _ = bob_dec.decrypt_to_string(encrypted, key_sequences)
    print(f"Bob's output: '{bob_strings[0][:50]}...'")

    print("\nEve attempting to break encryption without keys...")
    eve_strings, eve_tokens, _ = eve_att.attack_to_string(encrypted)
    print(f"Eve's output: '{eve_strings[0][:50]}...'")

    print("\n" + "="*70)

ENHANCED BOB DECODER & EVE ATTACKER

Bob attempting decryption with correct keys...
Bob's output: 'kkkkkkkkkkkk...'

Eve attempting to break encryption without keys...
Eve's output: 'sssssssssssssssssssssssssss$ssssssssssssssssssssss...'

